# **FRAMEWORKS DE DEEPLEARNING**
# **Prática - Keras**
**Autor**: Renan Santos Mendes

**Email**: renansantosmendes@gmail.com

**Descrição**: Este notebook apresenta um exemplo de uma rede neural profunda com mais de uma camada para um problema de classificação.


# **Saúde Fetal**

As Cardiotocografias (CTGs) são opções simples e de baixo custo para avaliar a saúde fetal, permitindo que os profissionais de saúde atuem na prevenção da mortalidade infantil e materna. O próprio equipamento funciona enviando pulsos de ultrassom e lendo sua resposta, lançando luz sobre a frequência cardíaca fetal (FCF), movimentos fetais, contrações uterinas e muito mais.

Este conjunto de dados contém 2126 registros de características extraídas de exames de Cardiotocografias, que foram então classificados por três obstetras especialistas em 3 classes:

- Normal
- Suspeito
- Patológico

In [ ]:
%%capture
!pip install autokeras==1.0.19 mlflow dagshub

# 1 - Importando os módulos necessários

In [ ]:
import os
import tensorflow
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, InputLayer, Dropout
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
from keras.regularizers import l1, l2

import random
import numpy as np
import random as python_random
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Definindo funções adicionais

In [ ]:
def plot_convergence(hist, metric):

  df = pd.DataFrame(hist.history)
  fig = px.line(df,
                x=np.arange(df[metric].shape[0]),
                y=[f'{metric}', f'val_{metric}'])
  fig.show()

def reset_seeds():
   os.environ['PYTHONHASHSEED']=str(42)
   tf.random.set_seed(42)
   np.random.seed(42)
   random.seed(42)

# 2 - Fazendo a leitura do dataset e atribuindo às respectivas variáveis

In [ ]:
data = pd.read_csv('https://raw.githubusercontent.com/renansantosmendes/lectures-cdas-2023/master/fetal_health.csv')

In [ ]:
data.head()

,baseline value,accelerations,fetal_movement,uterine_contractions,light_decelerations,severe_decelerations,prolongued_decelerations,abnormal_short_term_variability,mean_value_of_short_term_variability,percentage_of_time_with_abnormal_long_term_variability,...,histogram_min,histogram_max,histogram_number_of_peaks,histogram_number_of_zeroes,histogram_mode,histogram_mean,histogram_median,histogram_variance,histogram_tendency,fetal_health
0,120.0,0.000,0.0,0.000,0.000,0.0,0.0,73.0,0.5,43.0,...,62.0,126.0,2.0,0.0,120.0,137.0,121.0,73.0,1.0,2.0
1,132.0,0.006,0.0,0.006,0.003,0.0,0.0,17.0,2.1,0.0,...,68.0,198.0,6.0,1.0,141.0,136.0,140.0,12.0,0.0,1.0
2,133.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.1,0.0,...,68.0,198.0,5.0,1.0,141.0,135.0,138.0,13.0,0.0,1.0
3,134.0,0.003,0.0,0.008,0.003,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,11.0,0.0,137.0,134.0,137.0,13.0,1.0,1.0
4,132.0,0.007,0.0,0.008,0.000,0.0,0.0,16.0,2.4,0.0,...,53.0,170.0,9.0,0.0,137.0,136.0,138.0,11.0,1.0,1.0


In [ ]:
data.fetal_health.value_counts()

fetal_health
1.0    1655
2.0     295
3.0     176
Name: count, dtype: int64

# 3 - Preparando o dado antes de iniciar o treino do modelo

In [ ]:
data["fetal_health"]

0       2.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
2121    2.0
2122    2.0
2123    2.0
2124    2.0
2125    1.0
Name: fetal_health, Length: 2126, dtype: float64

In [ ]:
y

0       1.0
1       0.0
2       0.0
3       0.0
4       0.0
       ... 
2121    1.0
2122    1.0
2123    1.0
2124    1.0
2125    0.0
Name: fetal_health, Length: 2126, dtype: float64

In [ ]:
X = data.drop(["fetal_health"], axis=1)
y = data["fetal_health"] - 1

columns_names = list(X.columns)
X_df = preprocessing.StandardScaler().fit_transform(X)
X_df = pd.DataFrame(X_df, columns=columns_names)

X_train, X_test, y_train, y_test = train_test_split(X_df,
                                                    y,
                                                    test_size=0.3,
                                                    random_state=42)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
y_train

array([[1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [ ]:
output = y_test[0]
output

array([1., 0., 0.], dtype=float32)

In [ ]:
input = X_test.iloc[0].values
input

array([-0.03088439, -0.30488126,  0.01113035, -0.46391894,  0.03734947,
       -0.0574756 , -0.2687543 , -0.05760286, -0.26361916, -0.53536128,
        1.28176109, -0.03712546,  0.04806603, -0.00141582,  0.31600345,
       -0.45844382,  0.09451891,  0.02498176, -0.00624416, -0.33855066,
       -0.52452553])

In [ ]:
prediction = np.array([0.98, 0.01, 0.01])

In [ ]:
np.argmax(prediction)

0

In [ ]:
abs(output - prediction).sum()

0.39999999999999997

In [ ]:
abs(output - prediction).sum()

0.19999999999999996

In [ ]:
abs(output - prediction).sum()

0.04000000000000002

# 4 - Criando o modelo e adicionando as camadas

In [ ]:
from sklearn.svm import SVC

In [ ]:
svm = SVC()

In [ ]:
data_input_shape =X_train.iloc[0].shape

In [ ]:
reset_seeds()
model = Sequential(name='neural_network')
model.add(layer=InputLayer(input_shape=data_input_shape))
model.add(layer=Dense(units=20,
                      activation='relu'))
model.add(layer=Dense(units=20,
                      activation='relu'))
model.add(layer=Dense(units=20,
                      activation='relu'))
model.add(layer=Dense(units=3,
                      activation='softmax'))

In [ ]:
model.get_weights()[0].shape

(21, 20)

# 5 - Compilando o modelo


In [ ]:
model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

# 6 - Executando o treino do modelo

In [ ]:
%%time
hist = model.fit(x=X_train,
                 y=y_train,
                 epochs=30)

Epoch 1/30
47/47 [==============================] - 1s 3ms/step - loss: 0.7832 - accuracy: 0.7500
Epoch 2/30
47/47 [==============================] - 0s 2ms/step - loss: 0.5311 - accuracy: 0.8078
Epoch 3/30
47/47 [==============================] - 0s 2ms/step - loss: 0.4194 - accuracy: 0.8300
Epoch 4/30
47/47 [==============================] - 0s 3ms/step - loss: 0.3565 - accuracy: 0.8629
Epoch 5/30
47/47 [==============================] - 0s 3ms/step - loss: 0.3172 - accuracy: 0.8864
Epoch 6/30
47/47 [==============================] - 0s 3ms/step - loss: 0.2903 - accuracy: 0.8911
Epoch 7/30
47/47 [==============================] - 0s 2ms/step - loss: 0.2704 - accuracy: 0.8918
Epoch 8/30
47/47 [==============================] - 0s 2ms/step - loss: 0.2531 - accuracy: 0.9026
Epoch 9/30
47/47 [==============================] - 0s 2ms/step - loss: 0.2423 - accuracy: 0.9073
Epoch 10/30
47/47 [==============================] - 0s 2ms/step - loss: 0.2316 - accuracy: 0.9133
Epoch 11/30
47/47 [

In [ ]:
%%time
for i in range(150):
  print(np.argmax(model.predict(X_test.iloc[i].values.reshape(1,-1))))

1/1 [==============================] - 0s 25ms/step
0
1/1 [==============================] - 0s 21ms/step
0
1/1 [==============================] - 0s 25ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 21ms/step
0
1/1 [==============================] - 0s 22ms/step
2
1/1 [==============================] - 0s 20ms/step
0
1/1 [==============================] - 0s 30ms/step
2
1/1 [==============================] - 0s 23ms/step
0
1/1 [==============================] - 0s 22ms/step
1
1/1 [==============================] - 0s 27ms/step
0
1/1 [==============================] - 0s 23ms/step
2
1/1 [==============================] - 0s 21ms/step
0
1/1 [==============================] - 0s 25ms/step
0
1/1 [==============================] - 0s 25ms/step
0
1/1 [==============================] - 0s 22ms/step
1
1/1 [==============================] - 0s 22ms/step
0
1/1 [==============================] - 0s 23ms/step
0
1/1 [=======================

# 6.1 Avaliando o modelo

# 6.2 Plotando a acurácia do modelo

In [ ]:
...

# DagsHub
<img src="https://api.flatworld.co/wp-content/uploads/2020/10/DAGsHub-Logo.png" height=100>


DagsHub é uma plataforma de colaboração e compartilhamento de código voltada para a comunidade de Ciência de Dados e Machine Learning. É uma plataforma que permite que os desenvolvedores, cientistas de dados e pesquisadores compartilhem seus projetos, coletem feedback de outros usuários, colaborem em projetos em equipe e gerenciem seu fluxo de trabalho de desenvolvimento de forma mais eficiente.

A plataforma é baseada em Git e Git-LFS, o que significa que o controle de versão e o gerenciamento de arquivos são suportados nativamente. Além disso, o DagsHub oferece uma série de recursos adicionais que visam melhorar a experiência de desenvolvimento de projetos de Ciência de Dados e Machine Learning, como integração com ferramentas de treinamento de modelos, ambientes de execução, Jupyter notebooks e fluxos de trabalho de DAGs (gráficos acíclicos direcionados).


Com o DagsHub, os usuários podem compartilhar seus projetos publicamente ou limitar o acesso a equipes específicas, criando uma comunidade de colaboração de dados e Machine Learning mais ampla. A plataforma também incentiva a colaboração em projetos de código aberto, com recursos para facilitar a contribuição e a revisão de código pelos usuários.

Em resumo, o DagsHub é uma plataforma completa para compartilhamento, colaboração e gerenciamento de projetos de Ciência de Dados e Machine Learning, que tem como objetivo tornar o trabalho em equipe mais fácil e eficiente para os usuários.

### [Link](https://dagshub.com/) para acesso.

In [ ]:
%%capture
!pip install mlflow dagshub

In [ ]:
import mlflow
import dagshub

mlflow.tensorflow.autolog()
# dagshub.init("seu_repositorio","seu_username")

# **Vendo a arquitetura do modelo**

In [ ]:
from keras.utils import plot_model

In [ ]:
plot_model(model=model,
           show_shapes=True,
           show_dtype=True,
           show_layer_names=True,
           rankdir='LR', #'LR' or 'TB'
           expand_nested=False,
           dpi=96,
           layer_range=None,
           show_layer_activations=True,
           show_trainable=True)

# **CallBacks**

Callbacks no Keras são objetos que podem ser passados como parâmetros ao treinar um modelo de rede neural. Eles permitem que o modelo execute ações durante o treinamento em momentos específicos, como no início ou fim de uma época de treinamento, ou após cada lote de amostras ter sido processado.

In [ ]:
def get_compiled_model():
  reset_seeds()
  ...
  return model

## **ModelCheckpoint**

In [ ]:
from keras.callbacks import ModelCheckpoint

filepath = "weights.best.hdf5"
checkpoint = ...

model = get_compiled_model()

with mlflow.start_run(run_name='callbacks_1'):
  ...

## **EarlyStopping**

In [ ]:
from keras.callbacks import EarlyStopping

early_stopping = ...

model = get_compiled_model()

with mlflow.start_run(run_name='callbacks_2'):
 ...

# Corrigindo Overfitting

Overfitting é um conceito crucial em aprendizado de máquina e estatística que descreve a situação em que um modelo se ajusta tão bem aos dados de treinamento que perde sua capacidade de generalização para novos dados. Em outras palavras, o modelo "decora" os dados de treinamento em vez de aprender padrões subjacentes que se aplicam a dados não vistos. Isso resulta em um desempenho deficiente quando o modelo é testado em dados que não foram usados durante o treinamento.

Existem várias razões pelas quais o overfitting pode ocorrer:

- Modelo muito complexo: Um modelo excessivamente complexo pode capturar até mesmo o ruído nos dados de treinamento, o que não é representativo do verdadeiro relacionamento entre as variáveis.

- Dados insuficientes: Quando os dados de treinamento são escassos em relação à complexidade do modelo, o modelo pode aprender padrões específicos dos dados de treinamento que não se aplicam a novos dados.

- Falta de regularização: Técnicas de regularização, como penalização L1 ou L2, são frequentemente usadas para evitar overfitting. A ausência de tais técnicas pode permitir que o modelo se ajuste demais aos dados de treinamento.

Corrigir o overfitting é crucial para garantir que nosso modelo seja útil na prática

In [ ]:
reset_seeds()
model = Sequential()
model.add(Dense(100, input_shape=(21,), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(300, activation='relu'))
model.add(Dense(200, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='Adam',
              metrics=['accuracy'])
print(model.summary())
with mlflow.start_run(run_name='model_overfitting'):
  hist = model.fit(X_train,
                  y_train,
                  epochs=20,
                  validation_split=0.2,
                  verbose=3)

print(model.evaluate(X_test, y_test))

In [ ]:
plot_convergence(hist, 'loss')

# Regularização

a regularização é uma técnica usada para evitar o overfitting, adicionando uma penalidade aos pesos do modelo durante o processo de otimização. Existem diferentes tipos de regularização, mas duas das mais comuns são a regularização L1 e L2.

- Regularização L1 (Lasso): Na regularização L1, uma penalidade é adicionada à função de perda durante o treinamento, proporcional à magnitude absoluta dos pesos do modelo. Isso leva à "esparsidade" nos pesos, ou seja, alguns pesos podem se tornar exatamente zero, reduzindo a complexidade do modelo. No Keras, a regularização L1 pode ser aplicada usando a classe keras.regularizers.l1().

- Regularização L2 (Ridge): Na regularização L2, uma penalidade é adicionada à função de perda durante o treinamento, proporcional à magnitude ao quadrado dos pesos do modelo. Isso desencoraja pesos grandes, tornando o modelo mais suave e menos propenso ao overfitting. No Keras, a regularização L2 pode ser aplicada usando a classe keras.regularizers.l2().

Ambas as formas de regularização podem ser adicionadas como argumentos ao definir camadas em um modelo Keras.

### **L1 - Lasso**

### **L2 - Ridge**

# Dropout

O dropout é uma técnica de regularização popular para combater o overfitting em modelos de redes neurais. Durante o treinamento, o dropout desativa aleatoriamente um determinado número de unidades (neurônios) em uma camada, definindo suas saídas como zero. Isso significa que, temporariamente, essas unidades não contribuem para o processo de treinamento e não são atualizadas durante uma etapa de otimização. O dropout é aplicado durante o treinamento, mas não durante a inferência (ou seja, quando o modelo está fazendo previsões).

A ideia principal por trás do dropout é forçar a rede neural a aprender representações mais robustas e distribuídas dos dados, em vez de depender excessivamente de neurônios específicos ou correlações entre eles. Isso ajuda a reduzir a coadaptação entre os neurônios e, consequentemente, o overfitting.

No Keras, o dropout pode ser facilmente aplicado às camadas usando a classe Dropout no módulo keras.layers.

# **Modelo Funcional**

O modelo funcional no Keras é uma API flexível e poderosa que permite construir modelos de redes neurais com topologias mais complexas do que a API sequencial oferece. Enquanto o modelo sequencial é adequado para uma pilha simples de camadas, o modelo funcional permite a criação de modelos com grafos de computação mais elaborados, incluindo múltiplas entradas, saídas e caminhos de conexão.

A principal diferença entre o modelo funcional e o modelo sequencial é que, no modelo funcional, você define explicitamente as conexões entre as camadas. Isso permite que você crie arquiteturas de rede mais complexas, como modelos com múltiplas entradas ou saídas, compartilhamento de camadas e bifurcações.

O modelo funcional no Keras oferece uma maneira flexível e intuitiva de construir arquiteturas de redes neurais mais complexas, permitindo criar modelos adaptados a uma ampla variedade de problemas de aprendizado profundo.

In [ ]:
%%time
reset_seeds()
inputs = ...
fc1 = ...
fc2 = ...
outputs = ...

model = ...

model.summary()

model.compile(loss='categorical_crossentropy',
              metrics=['accuracy'])

hist = model.fit(X_train,
                 y_train,
                 epochs=10,
                 validation_split=0.2,
                 verbose=3)

# **Uso de objetos**

In [ ]:
reset_seeds()
model = Sequential()
model.add(InputLayer(input_shape=(21,)))
model.add(Dense(10, activation='relu' ))
model.add(Dense(10, activation='relu' ))
model.add(Dense(3, activation='softmax'))

lr_schedule = ...

opt = ...

model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

hist = model.fit(X_train,
                 y_train,
                 epochs=10,
                 validation_split=0.2,
                 verbose=3)

# **AutoML - AutoKeras**

AutoKeras é uma biblioteca de aprendizado de máquina automatizada (AutoML) de código aberto para Python, que permite a automatização de tarefas de seleção de modelo, pré-processamento de dados, ajuste de hiperparâmetros e treinamento de modelos de aprendizado de máquina de forma eficiente.

O AutoKeras utiliza técnicas de busca automática de hiperparâmetros, como a busca aleatória, busca em grade e otimização bayesiana, para encontrar a melhor configuração de modelo e hiperparâmetros para um determinado problema de aprendizado de máquina. Ele suporta uma variedade de tarefas de aprendizado de máquina, incluindo classificação, regressão, segmentação de imagens e processamento de linguagem natural.

Além disso, o AutoKeras possui uma interface fácil de usar que permite aos usuários treinar modelos sem a necessidade de conhecimentos avançados em aprendizado de máquina ou programação. Isso torna o AutoKeras uma ferramenta útil para usuários iniciantes e avançados que desejam economizar tempo e recursos ao treinar modelos de aprendizado de máquina de alta qualidade.

In [ ]:
from autokeras import StructuredDataClassifier

In [ ]:
...

In [ ]:
...